# CV Project Jupyter Script

## Clone Project From ( Optional )

如果是用 Colab 開啟此 GitHub 檔案的話，請將 PullFromRepo 設成 True，他會自動 Pull 對應 branch 的其他檔案下來（主要是 functional.py 以及 light_cnn.py 兩個檔案作為 dependency）。

如果是使用 git clone 整個專案到 local 上的話，則不需要 PullFromRepo（預設為此）。

In [1]:
PullFromRepo = False
DoDownload = False
ImageTrick = False

if PullFromRepo:
    !git init
    !git remote add origin "https://github.com/freshLiver/CV_PROJECT"
    !git pull origin py-test

    %pip install gdown


## Imports and Hyper-parameters

### Imports

下載資料集以及其他 training 所需的 packages

In [2]:
import gdown

import cv2
import math
import numpy as np
from os import system
from PIL import Image
from pathlib import Path
from matplotlib import pyplot as plt

import torch
import torchvision.transforms as transforms
from torch.utils.data import DataLoader


from functional import TrainingHelper, ImageList
from light_cnn import LightCNN_9Layers as LightCNN


### Hyper-parameters

訓練所需的參數以及資料集路徑設定

In [3]:
# training
EPOCHS = 10
BS = 256             # batch size
LR = 0.001          # learning rate
NUM_WORKER = 0
PRINT_FREQUENCY = 10
VALID_RATIO = 0.2


# give me abs path as ROOT(work dir)
ROOT = Path.home().joinpath("Downloads")

# dataset paths

# DATA_SRC = "https://drive.google.com/uc?id=11zQKShQ_qTt5HJtZDpkvskPXuDb2rbbo"
DATA_SRC = "https://drive.google.com/uc?id=11KFpKd8i8r1nES1AmSminorvRivB2M8_"

DATA_DST = ROOT.joinpath("vggface2-test.zip")
DATA_DIR = ROOT.joinpath("vggface2")


TRAIN_LIST = ROOT.joinpath("train_list.txt")
VALID_LIST = ROOT.joinpath("valid_list.txt")


## Data Pre-processing

### Download and Unzip Dataset ( Optional )

由於 vggface2 的 dataset 是放在我的 google drive 上，因此這邊會使用 gdown 下載資料集到前面設定的路徑（ROOT）下並解壓縮，若已經有下載 dataset 的話請在上面設定路徑，並不要執行這邊。

In [4]:
if DoDownload:

    # download dataset
    gdown.download(str(DATA_SRC), str(DATA_DST), False)

    # extract if dir not exists
    if not DATA_DIR.exists():
        system(f"unzip -d {DATA_DIR} {DATA_DST} > unzip.log")


### Image Trick (Optional)

直接對原始圖片進行一些特別的處理，並複寫原圖片以在 Dataset 讀寫時減少運算量。

In [6]:
if ImageTrick:

    # visit each dir under data dir
    for subdir in DATA_DIR.glob("*"):

        # visit all files under this dir
        for imgPath in subdir.glob("*"):

            # load img
            origin = cv2.imread(str(imgPath), cv2.IMREAD_GRAYSCALE)

            # do canny on original image
            edges = cv2.Canny(origin, 150, 150)
            res = cv2.bitwise_or(origin, edges)

            # overwrite original image
            cv2.imwrite(str(imgPath), res)


### Parse, Split and Save Dataset

讀取所有設定路徑下的圖片，並將各個 class 的圖片以及對應的 class 依據前面設定的比例分割成 training dataset 以及 validation dataset，並寫入到指定的檔案中（TRAIN_LIST 以及 VALID_LIST）。

In [ ]:
# load data into dict
train_data = []
valid_data = []


NUM_CLASSES = 0
CLASS_MAPPING = {}
for index, subdir in enumerate(DATA_DIR.glob("*")):

    # iterate each file in this dir
    image_list = []
    for imgPath in subdir.glob("*"):
        image_list.append((Path.relative_to(imgPath, ROOT), index))

    # split into train, validation list
    tSize = math.floor(len(image_list)*(1-VALID_RATIO))

    train_data += image_list[:tSize]
    valid_data += image_list[tSize:]

    # add label:class mapping
    CLASS_MAPPING[NUM_CLASSES] = subdir.name

    # count class
    NUM_CLASSES += 1

    # add NUM_CLASSES limit for faster testing
    # if NUM_CLASSES > 30:
    #     break


# save to file
with open(TRAIN_LIST, 'w') as f:
    for img, cat in train_data:
        f.write(f'{img} {cat}\n')

with open(VALID_LIST, 'w') as f:
    for img, cat in valid_data:
        f.write(f'{img} {cat}\n')


## Main Script

### Choose Model

In [ ]:
model = LightCNN(num_classes=NUM_CLASSES)
print(model)


### DataLoader

設定 Training 以及 Validation 所需的 transforms 以及 data loader。

In [ ]:
# define transforms
train_transform = transforms.Compose([
    transforms.Resize(128),
    transforms.RandomCrop(128),
    transforms.ToTensor(),
    transforms.Normalize([.5], [.5])
])

valid_transform = transforms.Compose([
    transforms.Resize(128),
    transforms.CenterCrop(128),
    transforms.ToTensor(),
    transforms.Normalize([.5], [.5])
])


# create data loaders
train_loader = DataLoader(
    ImageList(root=ROOT, fileList=TRAIN_LIST, transform=train_transform),
    batch_size=BS,
    shuffle=True,
    num_workers=NUM_WORKER,
    pin_memory=True
)

valid_loader = DataLoader(
    ImageList(root=ROOT, fileList=VALID_LIST, transform=valid_transform),
    batch_size=BS,
    shuffle=False,
    num_workers=NUM_WORKER,
    pin_memory=True
)


### Train !

開始 Train 以及 Validate，並紀錄每個 Epoch 的 avg loss 以及 avg accuracy。

In [ ]:

helper = TrainingHelper(
    train_dataloader=train_loader,
    valid_dataloader=valid_loader,
    epochs=EPOCHS,
    batch_size=BS,
    learning_rate=LR,
    print_frequency=PRINT_FREQUENCY,
    model=model,
    criterion=torch.nn.CrossEntropyLoss(),
    optimizer=torch.optim.Adam(params=model.parameters(), lr=LR)
)

for iEpoch in range(0, EPOCHS):

    # train for one epoch
    train_loss, train_acc = helper.train(iEpoch)

    # evaluate on validation set
    valid_loss, valid_acc = helper.validate(iEpoch)

    # log epoch result
    helper.LOGGER.push(train_loss, train_acc, valid_loss, valid_acc)


## Save Model and Show Result Graph ( Optional )

儲存 model 到指定的路徑，並使用 Matplotlib 顯示各個 epoch 的 loss 以及 accuracy 的走向。

In [ ]:
helper.LOGGER.visualize(loss_dst='loss.png', accuracy_dst='acc.png')
helper.LOGGER.save("log.json")

torch.save(model.state_dict(), "model.pth")
